In [1]:
from pyntcloud import PyntCloud
import open3d as o3d
import numpy as np
import laspy
from sklearn.decomposition import PCA


# FILTERING
pc_2 = PyntCloud.from_file("finalproject.ply")
coords_f = np.asarray(pc_2.points)

pcd_f = o3d.geometry.PointCloud()

nc = []

for p in coords_f:
      if p[0] > 576750 and p[0] < 576950 and p[1] > 7032500 and p[1] < 7032700:
        nc.append(p)
coords_f = np.asarray(nc)

pcd_f.points = o3d.utility.Vector3dVector(coords_f)
o3d.io.write_point_cloud("finalproject_segment.ply", pcd_f)



RuntimeError: 

In [ ]:
pcd = o3d.io.read_point_cloud("finalproject_segment.ply")


# Visualize Point Cloud
o3d.visualization.draw_geometries([pcd_f])


print("Statistical oulier removal")
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
o3d.visualization.draw_geometries([cl])

o3d.io.write_point_cloud("finalproject_wn.ply", cl)
pc = PyntCloud.from_file("finalproject_wn.ply")
pcd_array = np.asarray(pc.points)

In [ ]:
# Voxelization
voxelgrid_id = pc.add_structure("voxelgrid", size_x=2, size_y=2, size_z=2)
voxelgrid = pc.structures[voxelgrid_id]
voxels = voxelgrid.voxel_n

print("Number of voxels: ", len(voxels))
print("Voxel IDs: ", voxels)

# Voxels containing points
occupied_voxels = np.unique(voxelgrid.voxel_n)
print ("Number of occupied voxels: ", len(occupied_voxels))

# Visualize voxels
voxelgrid.plot(d=3, mode="density", cmap="hsv")

In [1]:
#Principal Components Analysis
pca = PCA(n_components=3)

roofs = np.zeros((1,3))
trees = np.zeros((1,3))
terrain = np.zeros((1,3))
uncategorized = np.zeros((1,3))

# Feature detection
for voxel in occupied_voxels:
    tmp = np.where(voxelgrid.voxel_n == voxel)
    voxel_points = pcd_array[tmp]



    if len(tmp[0]) > 2:
        pca.fit(voxel_points)

        ev = pca.explained_variance_ratio_  # Eigenvalues (explained_variance_ratio_)

        # Shape features
        l1, l2, l3 = ev
        L = (l1 - l2) / l1  # Linearity, verdi mellom 0-1, 1D, tree
        P = (l2 - l3) / l1  # Planarity, 2D, terreng
        S = l3 / l1  # Scattering, sphericity, 3D, bygning.
        O = (l1 * l2 * l3) ** (1 / 3)  # Omnivariance
        sum_ev = l1 + l2 + l3  # Sum of eigenvalues, denne er alltid 1, så ikke nå vits å bruke denne
        A = (l1 - l3) / l1  # Anisotropy
        #E = -1 * ((l1 * np.log(l1)) + (l2 * np.log(l2)) + (l3 * np.log(l3)))  # Eigenentropy
        change_curvature = l3 / sum_ev  # Change of curvature
        shape_features = [L, P, S, O, sum_ev, A, change_curvature] #Removed E 
        
        
        
        #if S > L:
        #    roofs = np.append(roofs, voxel_points, axis=0)
            
        
        if S > 0.1 and L > 0.1: #and L > P and L > S:
            trees = np.append(trees, voxel_points, axis=0)
            
        
        if P > L:
            terrain = np.append(terrain, voxel_points, axis=0)
        
        
        
        
        #else:
           # uncategorized = np.append(uncategorized, voxel_points, axis=0)
        
        
trees = np.delete(trees, 0, axis=0)  #grønn
pcd_tree = o3d.geometry.PointCloud()
pcd_tree.points = o3d.utility.Vector3dVector(trees)
pcd_tree.paint_uniform_color([0, 0.5, 0.3])
 
roofs = np.delete(roofs, 0, axis=0)   #rød      
pcd_roof = o3d.geometry.PointCloud()
pcd_roof.points = o3d.utility.Vector3dVector(roofs) 
pcd_roof.paint_uniform_color([1, 0, 0])

terrain = np.delete(terrain, 0, axis=0)   #gul
pcd_terrain = o3d.geometry.PointCloud()
pcd_terrain.points = o3d.utility.Vector3dVector(terrain)
pcd_terrain.paint_uniform_color([0.9, 0.7, 0])

#uncategorized = np.delete(uncategorized, 0, axis=0) #blå
#pcd_uncategorized = o3d.geometry.PointCloud()
#pcd_uncategorized = o3d.utility.Vector3dVector(uncategorized)
#pcd_uncategorized.paint_uniform_color([0, 0, 1])

o3d.visualization.draw_geometries([pcd_tree, pcd_roof, pcd_terrain])

NameError: name 'sd' is not defined